In [39]:
# Data handling libraries
import pandas as pd
pd.options.display.max_rows = 10000
import numpy as np
import boto3
import io
import requests as req

# Initialize S3 client, location of files for this project
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

s3_bucket = "wri-public-data"

WB_DATA = "resourcewatch/world_bank_data_long_and_wide/"
CONVERSIONS = "resourcewatch/blog_data/GHG-GDP_Divergence_D3/Conversions/"

# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer)
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())

In [14]:
## World Bank data series codes and names

data_names_and_codes = {'EG.ELC.ACCS.ZS': 'Access to electricity (% of population)',
 'EG.FEC.RNEW.ZS': 'Renewable energy consumption (% of total final energy consumption)',
 'IT.NET.USER.ZS': 'Individuals using the Internet (% of population)',
 'NE.CON.PRVT.PC.KD': 'Household final consumption expenditure per capita (constant 2010 US$)',
 'NV.IND.TOTL.KD': 'Industry, value added (constant 2010 US$)',
 'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)',
 'SG.GEN.PARL.ZS': 'Proportion of seats held by women in national parliaments (%)',
 'SL.EMP.TOTL.SP.ZS': 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
 'SM.POP.NETM': 'Net migration',
 'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
 'SP.URB.TOTL.IN.ZS': 'Urban population (% of total)',
 'TM.VAL.MRCH.CD.WT': 'Merchandise imports (current US$)',
 'NY.GDP.MKTP.CD': 'GDP (current US$)'}

In [17]:
# Load conversions from wb_name to iso3
wb_name_to_iso3_conversion = read_from_S3(s3_bucket, CONVERSIONS+"World Bank to ISO3 name conversion.csv")

# Provide function to map from wb_name to ISO3
def add_iso(name):
    try:
        return(wb_name_to_iso3_conversion.loc[name,"ISO"])
    except:
        return(np.nan)

In [24]:
wb_name_to_iso3_conversion

,ISO
Afghanistan,AFG
Albania,ALB
Algeria,DZA
American Samoa,ASM
Andorra,AND
Angola,AGO
Anguila,AIA
Antigua and Barbuda,ATG
Argentina,ARG
Armenia,ARM


## Data in long form

In [21]:
indicators = list(data_names_and_codes.keys())
seed = indicators[0]
print(seed)
res = req.get("http://api.worldbank.org/countries/all/indicators/{}?format=json&per_page=10000".format(seed))
#print(res.text)
data = pd.io.json.json_normalize(res.json()[1])
data = data[["country.value", "date", "value"]]
value_name = data_names_and_codes[seed]
data.columns = ["Country Name", "Year", value_name]
data = data.set_index(["Country Name", "Year"])
all_world_bank_data = data

for indicator in indicators[1:]:
    # Results are paginated
    print(indicator)
    res = req.get("http://api.worldbank.org/countries/all/indicators/{}?format=json&per_page=10000".format(indicator))
    #print(res.text)
    data = pd.io.json.json_normalize(res.json()[1])
    data = data[["country.value", "date", "value"]]
    value_name = data_names_and_codes[indicator]
    data.columns = ["Country Name", "Year", value_name]
    data = data.set_index(["Country Name", "Year"])
    all_world_bank_data = all_world_bank_data.join(data, how="outer")
    
all_world_bank_data = all_world_bank_data.reset_index()
all_world_bank_data["ISO3"] = list(map(add_iso, all_world_bank_data["Country Name"]))
all_world_bank_data = all_world_bank_data.loc[pd.notnull(all_world_bank_data["ISO3"])]
all_world_bank_data = all_world_bank_data.set_index(["Country Name", "Year"])

EG.ELC.ACCS.ZS
EG.FEC.RNEW.ZS
IT.NET.USER.ZS
NE.CON.PRVT.PC.KD
NV.IND.TOTL.KD
NY.GDP.TOTL.RT.ZS
SG.GEN.PARL.ZS
SL.EMP.TOTL.SP.ZS
SM.POP.NETM
SP.DYN.LE00.IN
SP.URB.TOTL.IN.ZS
TM.VAL.MRCH.CD.WT
NY.GDP.MKTP.CD


In [23]:
all_world_bank_data.head()

Access to electricity (% of population)  \
Country Name   Year                                           
European Union 2016                                    None   
               2015                                    None   
               2014                                     100   
               2013                                     100   
               2012                                     100   

                    Renewable energy consumption (% of total final energy consumption)  \
Country Name   Year                                                                      
European Union 2016                                               None                   
               2015                                               None                   
               2014                                    16.053026214829                   
               2013                                   15.1442661475667                   
               2012                                   14.3811492809133                   

                    Individuals using the Internet (% of population)  \
Country Name   Year                                                    
European Union 2016                                 80.7691050337866   
               2015                                 78.4329000006293   
               2014                                 77.1139914118683   
               2013                                 75.5428368269042   
               2012                                 73.7061519154293   

                    Household final consumption expenditure per capita (constant 2010 US$)  \
Country Name   Year                                                                          
European Union 2016                                   19968.1369826023                       
               2015                                   19583.8758139882                       
               2014                                   19245.8917008979                       
               2013                                   19082.6827482441                       
               2012                                   19160.1520852802                       

                    Industry, value added (constant 2010 US$)  \
Country Name   Year                                             
European Union 2016                          3988821725434.18   
               2015                          3933732615792.92   
               2014                          3807777737722.81   
               2013                          3731658584714.89   
               2012                          3776290013765.67   

                    Total natural resources rents (% of GDP)  \
Country Name   Year                                            
European Union 2016                                     None   
               2015                        0.209417246513485   
               2014                        0.319622359791876   
               2013                        0.373596153789426   
               2012                        0.441742736714659   

                    Proportion of seats held by women in national parliaments (%)  \
Country Name   Year                                                                 
European Union 2016                                    28.418549063461              
               2015                                   28.3813206756945              
               2014                                   27.7298367973218              
               2013                                   27.0906547133931              
               2012                                   25.7243875884594              

                    Employment to population ratio, 15+, total (%) (modeled ILO estimate)  \
Country Name   Year                                                                         
European Union 2016                                   52.3869840451532                      
   

In [41]:
for code, name in data_names_and_codes.items():
    long_form = all_world_bank_data[name]
    long_form = long_form.reset_index()
    long_form = long_form[pd.notnull(long_form[name])]
    write_to_S3(long_form, s3_bucket, WB_DATA + "wb_data_long_{}.csv".format(name.replace(" ", "_")))

## Data in wide form

In [49]:
df = all_world_bank_data.reset_index()
index_by_countryName = df.columns[0]
columns_by_year = df.columns[1]

# Only go up to -1 b/c we don't need to do this for the ISO column
names = df.columns[2:-1]

for name in names:
    wide_form = df.pivot(index=index_by_countryName,
           columns=columns_by_year,
           values=name)
    wide_form = wide_form.dropna(how="all", axis=1)
    wide_form = wide_form.reset_index()
    wide_form["ISO3"] = list(map(add_iso, wide_form["Country Name"]))
    wide_form = wide_form[pd.notnull(wide_form["ISO3"])]
    # Carto doesn't except column names that are numbers! 
    wide_form.columns = ["yr_" + str(col) for col in wide_form.columns]
    write_to_S3(wide_form, s3_bucket, WB_DATA + "wb_data_wide_{}.csv".format(name.replace(" ", "_")))

## World Bank country names to delete

Arab World, Middle income, Europe & Central Asia (IDA & IBRD countries), IDA total, Latin America & the Caribbean (IDA & IBRD countries), Middle East & North Africa (IDA & IBRD countries), *blank* (ID 268), Europe & Central Asia (excluding high income), IBRD only, IDA only, Early-demographic dividend, Latin America & the Caribbean (excluding high income), Middle East & North Africa, Middle East & North Africa (excluding high income), Late-demographic dividend, Pacific island small states, Europe & Central Asia, European Union, High income, IDA & IBRD total, IDA blend, Caribbean small states, Central Europe and the Baltics, East Asia & Pacific, East Asia & Pacific  (excluding high income), Low & middle income, Lower middle income, Other small states, Latin America & Caribbean, East Asia & Pacific (IDA & IBRD countries), Euro area, OECD members, North America, Middle East & North Africa (excluding high income),  Post-demographic dividend, Small states, South Asia, Upper middle income, World, heavily indebted poor countries (HIPC), Least developed countries: UN classification, *blank* (ID 267), *blank* (ID 265), Latin America & Caribbean, IDA & IBRD total, IBRD only, Europe & Central Asia, sub-Saharan Africa (excluding high income), Macao SAR China,  sub-Saharan Africa, pre-demographic dividend, South Asia (IDA & IBRD), sub-Saharan Africa (IDA & IBRD), Upper middle income, fragile and conflict affected 